### Tree Methods

In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('722TreeMethods').getOrCreate()

# import other packages
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.types import (StructField,StringType,IntegerType,StructType)

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/13 00:21:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/13 00:21:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


 Load Data

In [2]:
# integrate the 2 sources
# code adapted from https://www.geeksforgeeks.org/merge-two-dataframes-in-pyspark/
import functools 
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

In [3]:
# create a variable with the correct structure
data_schema = [StructField('Total Household Income',IntegerType(),True), StructField('Region',StringType(),True),
              StructField('Total Food Expenditure',IntegerType(),True), StructField('Main Source of Income',StringType(),True),
              StructField('Agricultural Household indicator',IntegerType(),True), StructField('Bread and Cereals Expenditure',IntegerType(),True),
              StructField('Total Rice Expenditure',IntegerType(),True), StructField('Meat Expenditure',IntegerType(),True),
              StructField('Total Fish and  marine products Expenditure',IntegerType(),True), StructField('Fruit Expenditure',IntegerType(),True),
              StructField('Vegetables Expenditure',IntegerType(),True), StructField('Restaurant and hotels Expenditure',IntegerType(),True),
              StructField('Alcoholic Beverages Expenditure',IntegerType(),True), StructField('Tobacco Expenditure',IntegerType(),True),
              StructField('Clothing, Footwear and Other Wear Expenditure',IntegerType(),True), StructField('Housing and water Expenditure',IntegerType(),True),
              StructField('Imputed House Rental Value',IntegerType(),True), StructField('Medical Care Expenditure',IntegerType(),True),
              StructField('Transportation Expenditure',IntegerType(),True), StructField('Communication Expenditure',IntegerType(),True),
              StructField('Education Expenditure',IntegerType(),True), StructField('Miscellaneous Goods and Services Expenditure',IntegerType(),True),
              StructField('Special Occasions Expenditure',IntegerType(),True), StructField('Crop Farming and Gardening expenses',IntegerType(),True),
              StructField('Total Income from Entrepreneurial Acitivites',IntegerType(),True), StructField('Household Head Sex',StringType(),True),
              StructField('Household Head Age',IntegerType(),True), StructField('Household Head Marital Status',StringType(),True),
              StructField('Household Head Highest Grade Completed',StringType(),True), StructField('Household Head Job or Business Indicator',StringType(),True),
              StructField('Household Head Occupation',StringType(),True), StructField('Household Head Class of Worker',StringType(),True),
              StructField('Type of Household',StringType(),True), StructField('Total Number of Family members',IntegerType(),True),
              StructField('Members with age less than 5 year old',IntegerType(),True), StructField('Members with age 5 - 17 years old',IntegerType(),True),
              StructField('Total number of family members employed',IntegerType(),True), StructField('Type of Building/House',StringType(),True),
              StructField('Type of Roof',StringType(),True), StructField('Type of Walls',StringType(),True),
              StructField('House Floor Area',IntegerType(),True), StructField('House Age',IntegerType(),True),
              StructField('Number of bedrooms',IntegerType(),True), StructField('Tenure Status',StringType(),True),
              StructField('Toilet Facilities',StringType(),True), StructField('Electricity',IntegerType(),True),
              StructField('Main Source of Water Supply',StringType(),True), StructField('Number of Television',IntegerType(),True),
              StructField('Number of CD/VCD/DVD',IntegerType(),True), StructField('Number of Component/Stereo set',IntegerType(),True),
              StructField('Number of Refrigerator/Freezer',IntegerType(),True), StructField('Number of Washing Machine',IntegerType(),True),
              StructField('Number of Airconditioner',IntegerType(),True), StructField('Number of Car, Jeep, Van',IntegerType(),True),
              StructField('Number of Landline/wireless telephones',IntegerType(),True), StructField('Number of Cellular phone',IntegerType(),True),
              StructField('Number of Personal Computer',IntegerType(),True), StructField('Number of Stove with Oven/Gas Range',IntegerType(),True),
              StructField('Number of Motorized Banca',IntegerType(),True), StructField('Number of Motorcycle/Tricycle',IntegerType(),True)
              ]

final_struct = StructType(fields=data_schema)

In [4]:
# And now we can read in the data using that schema. The other fields are read as integer. 
df1 = spark.read.option("header", True).csv('dataset/data_ncr.csv', schema=final_struct)
df2 = spark.read.option("header", True).csv('dataset/data_therest.csv', schema=final_struct)
statsdata = unionAll([df1, df2])

print("Combined dataset with updated dtypes:", statsdata.count(), "x", len(statsdata.columns))
statsdata.printSchema()

Combined dataset with updated dtypes: 41544 x 60
root
 |-- Total Household Income: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Total Food Expenditure: integer (nullable = true)
 |-- Main Source of Income: string (nullable = true)
 |-- Agricultural Household indicator: integer (nullable = true)
 |-- Bread and Cereals Expenditure: integer (nullable = true)
 |-- Total Rice Expenditure: integer (nullable = true)
 |-- Meat Expenditure: integer (nullable = true)
 |-- Total Fish and  marine products Expenditure: integer (nullable = true)
 |-- Fruit Expenditure: integer (nullable = true)
 |-- Vegetables Expenditure: integer (nullable = true)
 |-- Restaurant and hotels Expenditure: integer (nullable = true)
 |-- Alcoholic Beverages Expenditure: integer (nullable = true)
 |-- Tobacco Expenditure: integer (nullable = true)
 |-- Clothing, Footwear and Other Wear Expenditure: integer (nullable = true)
 |-- Housing and water Expenditure: integer (nullable = true)
 |-- Impute

Clean Data

In [5]:
# Drop columns
Columns_To_Remove = ['Region', 'Number of Television', 'Number of CD/VCD/DVD', 'Number of Component/Stereo set', 
                     'Number of Refrigerator/Freezer', 'Number of Washing Machine', 'Number of Airconditioner', 
                     'Number of Car, Jeep, Van', 'Number of Landline/wireless telephones', 'Number of Cellular phone', 
                     'Number of Personal Computer', 'Number of Stove with Oven/Gas Range', 'Number of Motorized Banca', 
                     'Number of Motorcycle/Tricycle']
statsupdate = statsdata.drop(*Columns_To_Remove)

# Add New Column with Total Expenses
expenses_cols = ['Total Food Expenditure', 'Bread and Cereals Expenditure', 'Total Rice Expenditure', 'Meat Expenditure', 'Total Fish and  marine products Expenditure', 'Fruit Expenditure', 'Vegetables Expenditure', 'Restaurant and hotels Expenditure', 'Alcoholic Beverages Expenditure', 'Tobacco Expenditure', 'Clothing, Footwear and Other Wear Expenditure', 'Housing and water Expenditure', 'Medical Care Expenditure', 'Transportation Expenditure', 'Communication Expenditure', 'Education Expenditure', 'Miscellaneous Goods and Services Expenditure', 'Special Occasions Expenditure', 'Crop Farming and Gardening expenses']
statsupdate = statsupdate.withColumn('Total Expenditures', sum(statsupdate[col] for col in expenses_cols))

# Drop the other expenses columns, except Medical Care Expenditure
expenses_cols.remove('Medical Care Expenditure')
statsupdate = statsupdate.drop(*expenses_cols)
print("Updated dimensions after removing",len(expenses_cols)+len(Columns_To_Remove),"columns:",statsupdate.count(), "x", len(statsupdate.columns))

# remove na/nulls
statsupdate = statsupdate.na.fill('Not Specified')

# remove outliers
# calculate upper and lower bounds for outliers, adapted from internet
def calculate_bounds(df):
    bounds = {
        c: dict(
            zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0))
        )
        for c,d in zip(df.columns, df.dtypes) if d[1] == "int"
    }
    for c in bounds:
        iqr = bounds[c]['q3'] - bounds[c]['q1']
        bounds[c]['min'] = bounds[c]['q1'] - (iqr * 1.5)
        bounds[c]['max'] = bounds[c]['q3'] + (iqr * 1.5)
    return bounds
bounds = calculate_bounds(statsupdate)

# remove outliers for "Total Household Income"
c = "Total Household Income"
newstats = statsupdate.filter(F.col(c).between(bounds[c]['min'], bounds[c]['max']))

# remove outliers for "Medical Care Expenditure"
c = "Medical Care Expenditure"
newstats = newstats.filter(F.col(c).between(bounds[c]['min'], bounds[c]['max']))

# rename misspelled column
newstats = newstats.withColumnRenamed("Total Income from Entrepreneurial Acitivites", "Total Income from Entrepreneurial Activities")

# categorize
#create a list of the columns that are string types
categoricalColumns = [item[0] for item in newstats.dtypes if item[1].startswith('string') ]

#define a list of stages in your pipeline. The string indexer will be one stage
stages = []
#iterate through all categorical values, create a string indexer, assign new column name with 'Index' at end
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + ' Index')
    stages += [stringIndexer]

pipeline = Pipeline(stages = stages)
newstats = pipeline.fit(newstats).transform(newstats)

newstats.printSchema()

Updated dimensions after removing 32 columns: 41544 x 29


root
 |-- Total Household Income: integer (nullable = true)
 |-- Main Source of Income: string (nullable = false)
 |-- Agricultural Household indicator: integer (nullable = true)
 |-- Imputed House Rental Value: integer (nullable = true)
 |-- Medical Care Expenditure: integer (nullable = true)
 |-- Total Income from Entrepreneurial Activities: integer (nullable = true)
 |-- Household Head Sex: string (nullable = false)
 |-- Household Head Age: integer (nullable = true)
 |-- Household Head Marital Status: string (nullable = false)
 |-- Household Head Highest Grade Completed: string (nullable = false)
 |-- Household Head Job or Business Indicator: string (nullable = false)
 |-- Household Head Occupation: string (nullable = false)
 |-- Household Head Class of Worker: string (nullable = false)
 |-- Type of Household: string (nullable = false)
 |-- Total Number of Family members: integer (nullable = true)
 |-- Members with age less than 5 year old: integer (nullable = true)
 |-- Members wit

### Run Tree Methods Classifiers

In [6]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier


Prepare Data Formats

In [7]:
assembler = VectorAssembler(
    inputCols=["Total Household Income", "Main Source of Income Index", 
            "Agricultural Household indicator","Imputed House Rental Value",
            "Total Income from Entrepreneurial Activities",
            "Household Head Age", "Total Number of Family members",
            "Members with age less than 5 year old","Members with age 5 - 17 years old",
            "Total number of family members employed", "House Floor Area",
             "House Age","Number of bedrooms",
             "Electricity","Total Expenditures",
             "Main Source of Income Index","Household Head Sex Index",
             "Household Head Marital Status Index","Household Head Highest Grade Completed Index",
             "Household Head Job or Business Indicator Index",
             "Household Head Class of Worker Index","Type of Household Index",
             "Type of Building/House Index","Type of Roof Index","Type of Walls Index",
             "Tenure Status Index","Toilet Facilities Index",
             "Main Source of Water Supply Index"],
    outputCol="features")
output = assembler.transform(newstats)

labelIndexer = StringIndexer(inputCol = "Medical Care Expenditure", outputCol = "Medical Care Expenditure Label")

output = labelIndexer.fit(output).transform(output)


In [8]:
# Let's select the two columns we want. Features (which contains vectors), and the predictor.
final_data = output.select("features",'Medical Care Expenditure Label')

In [9]:
final_data.printSchema()
final_data.head()

root
 |-- features: vector (nullable = true)
 |-- Medical Care Expenditure Label: double (nullable = false)



22/10/13 00:22:04 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(features=SparseVector(28, {0: 301900.0, 3: 72000.0, 5: 73.0, 6: 4.0, 9: 1.0, 10: 25.0, 11: 15.0, 12: 2.0, 13: 1.0, 14: 381306.0, 16: 1.0, 17: 1.0, 18: 1.0, 19: 1.0, 20: 2.0, 21: 1.0}), Medical Care Expenditure Label=54.0)

In [10]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

Create and Train Models

In [11]:
# create all 3 models
# Use defaults to make the comparison "fair". This simplifies the comparison process.
dtc = DecisionTreeClassifier(labelCol='Medical Care Expenditure Label',featuresCol='features', maxBins=46)
rfc = RandomForestClassifier(labelCol='Medical Care Expenditure Label',featuresCol='features',  maxBins=46)
# gbt = GBTClassifier(labelCol='Medical Care Expenditure Label',featuresCol='features',  maxBins=46)

In [12]:
# Train the models (it's three models, so it might take some time).
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
#gbt_model = gbt.fit(train_data)

22/10/13 00:22:21 WARN DAGScheduler: Broadcasting large task binary with size 1531.5 KiB
22/10/13 00:22:25 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/10/13 00:22:31 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
22/10/13 00:22:41 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB
22/10/13 00:22:42 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
22/10/13 00:22:47 WARN DAGScheduler: Broadcasting large task binary with size 5.5 MiB
22/10/13 00:22:48 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/10/13 00:22:53 WARN DAGScheduler: Broadcasting large task binary with size 8.0 MiB
22/10/13 00:22:54 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/10/13 00:22:59 WARN DAGScheduler: Broadcasting large task binary with size 9.4 MiB
22/10/13 00:23:01 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/10/13 00:23:05 WARN DAGScheduler: Broadcasting l

In [13]:
# Model Comparison - compare each of these models
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
# gbt_predictions = gbt_model.transform(test_data)

In [14]:
# Let's import the evaluator.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [15]:
# Select (prediction, true label) and compute test error. 
acc_evaluator = MulticlassClassificationEvaluator(labelCol="Medical Care Expenditure Label", predictionCol="prediction", metricName="accuracy")

In [16]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
#gbt_acc = acc_evaluator.evaluate(gbt_predictions)

22/10/13 00:23:57 WARN DAGScheduler: Broadcasting large task binary with size 1965.4 KiB
22/10/13 00:24:02 WARN DAGScheduler: Broadcasting large task binary with size 49.5 MiB


In [17]:
# Let's do something a bit more complex in terms of printing, just so it's formatted nicer. 
print("Here are the results!")
print('-'*40)
print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*40)
print('A random forest ensemble has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*40)
#print('An ensemble using GBT has an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

Here are the results!
----------------------------------------
A single decision tree has an accuracy of: 4.32%
----------------------------------------
A random forest ensemble has an accuracy of: 4.30%
----------------------------------------


In [18]:
dtc_predictions.show()

22/10/13 00:24:34 WARN DAGScheduler: Broadcasting large task binary with size 1964.2 KiB


+--------------------+------------------------------+--------------------+--------------------+----------+
|            features|Medical Care Expenditure Label|       rawPrediction|         probability|prediction|
+--------------------+------------------------------+--------------------+--------------------+----------+
|(28,[0,1,2,3,4,5,...|                        6007.0|[623.0,114.0,132....|[0.03069721606306...|       0.0|
|(28,[0,1,2,3,4,5,...|                        1969.0|[623.0,114.0,132....|[0.03069721606306...|       0.0|
|(28,[0,1,2,3,4,5,...|                         423.0|[623.0,114.0,132....|[0.03069721606306...|       0.0|
|(28,[0,1,2,3,4,5,...|                        1776.0|[623.0,114.0,132....|[0.03069721606306...|       0.0|
|(28,[0,1,2,3,4,5,...|                        3389.0|[623.0,114.0,132....|[0.03069721606306...|       0.0|
|(28,[0,1,2,3,4,5,...|                        1086.0|[623.0,114.0,132....|[0.03069721606306...|       0.0|
|(28,[0,1,2,3,4,5,...|               

In [19]:
rfc_predictions.show()

22/10/13 00:24:36 WARN DAGScheduler: Broadcasting large task binary with size 49.5 MiB


+--------------------+------------------------------+--------------------+--------------------+----------+
|            features|Medical Care Expenditure Label|       rawPrediction|         probability|prediction|
+--------------------+------------------------------+--------------------+--------------------+----------+
|(28,[0,1,2,3,4,5,...|                        6007.0|[0.46197961353058...|[0.02309898067652...|       0.0|
|(28,[0,1,2,3,4,5,...|                        1969.0|[0.47998746087590...|[0.02399937304379...|       0.0|
|(28,[0,1,2,3,4,5,...|                         423.0|[0.46197961353058...|[0.02309898067652...|       0.0|
|(28,[0,1,2,3,4,5,...|                        1776.0|[0.46197961353058...|[0.02309898067652...|       0.0|
|(28,[0,1,2,3,4,5,...|                        3389.0|[0.46197961353058...|[0.02309898067652...|       0.0|
|(28,[0,1,2,3,4,5,...|                        1086.0|[0.48736449608540...|[0.02436822480427...|       0.0|
|(28,[0,1,2,3,4,5,...|               

In [20]:
# re-run
def reRunTreesClassifiers(final_data):
    # split the data
    train_data,test_data = final_data.randomSplit([0.7,0.3])
    # load the models
    dtc = DecisionTreeClassifier(labelCol='Medical Care Expenditure Label',featuresCol='features', maxBins=46)
    rfc = RandomForestClassifier(labelCol='Medical Care Expenditure Label',featuresCol='features',  maxBins=46)
    # train the data
    dtc_model = dtc.fit(train_data)
    rfc_model = rfc.fit(train_data)
    # predict 
    dtc_predictions = dtc_model.transform(test_data)
    rfc_predictions = rfc_model.transform(test_data)
    # measure accuracy
    dtc_acc = acc_evaluator.evaluate(dtc_predictions)
    rfc_acc = acc_evaluator.evaluate(rfc_predictions)
    print("Here are the results!")
    print('-'*40)
    print('A single decision tree has an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
    print('-'*40)
    print('A random forest ensemble has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
    print('-'*40)

In [21]:
reRunTreesClassifiers(output)

22/10/13 00:24:47 WARN DAGScheduler: Broadcasting large task binary with size 1564.5 KiB
22/10/13 00:24:52 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
22/10/13 00:24:58 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
22/10/13 00:25:09 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB
22/10/13 00:25:11 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
22/10/13 00:25:14 WARN DAGScheduler: Broadcasting large task binary with size 5.5 MiB
22/10/13 00:25:16 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/10/13 00:25:20 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
22/10/13 00:25:22 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/10/13 00:25:27 WARN DAGScheduler: Broadcasting large task binary with size 9.7 MiB
22/10/13 00:25:29 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/10/13 00:25:34 WARN DAGScheduler: Broadcasting l

Here are the results!
----------------------------------------
A single decision tree has an accuracy of: 4.14%
----------------------------------------
A random forest ensemble has an accuracy of: 4.20%
----------------------------------------


In [22]:
# decrease features even more
assembler = VectorAssembler(
    inputCols=["Total Household Income", 
                "Household Head Sex Index",
                "Household Head Marital Status Index"],
    outputCol="features")
output_less = assembler.transform(newstats)

labelIndexer = StringIndexer(inputCol = "Medical Care Expenditure", outputCol = "Medical Care Expenditure Label")
output_less = labelIndexer.fit(output_less).transform(output_less)
final_data_less = output_less.select("features",'Medical Care Expenditure Label')

In [23]:
# rerun
reRunTreesClassifiers(output_less)

22/10/13 00:27:14 WARN DAGScheduler: Broadcasting large task binary with size 1544.6 KiB
22/10/13 00:27:15 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
22/10/13 00:27:20 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB
22/10/13 00:27:21 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
22/10/13 00:27:22 WARN DAGScheduler: Broadcasting large task binary with size 9.7 MiB
22/10/13 00:27:23 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
22/10/13 00:27:25 WARN DAGScheduler: Broadcasting large task binary with size 18.8 MiB
22/10/13 00:27:28 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
22/10/13 00:27:33 WARN DAGScheduler: Broadcasting large task binary with size 21.0 MiB
22/10/13 00:27:35 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
22/10/13 00:27:40 WARN DAGScheduler: Broadcasting large task binary with size 22.2 MiB
22/10/13 00:27:42 WARN DAGScheduler: Broadcastin

Here are the results!
----------------------------------------
A single decision tree has an accuracy of: 3.84%
----------------------------------------
A random forest ensemble has an accuracy of: 3.86%
----------------------------------------


In [24]:
reRunTreesClassifiers(output_less)

22/10/13 00:29:26 WARN DAGScheduler: Broadcasting large task binary with size 1544.5 KiB
22/10/13 00:29:26 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
22/10/13 00:29:32 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB
22/10/13 00:29:33 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
22/10/13 00:29:34 WARN DAGScheduler: Broadcasting large task binary with size 9.7 MiB
22/10/13 00:29:35 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
22/10/13 00:29:38 WARN DAGScheduler: Broadcasting large task binary with size 19.8 MiB
22/10/13 00:29:40 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
22/10/13 00:29:45 WARN DAGScheduler: Broadcasting large task binary with size 22.5 MiB
22/10/13 00:29:47 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB
22/10/13 00:29:52 WARN DAGScheduler: Broadcasting large task binary with size 20.7 MiB
22/10/13 00:29:54 WARN DAGScheduler: Broadcastin

Here are the results!
----------------------------------------
A single decision tree has an accuracy of: 4.09%
----------------------------------------
A random forest ensemble has an accuracy of: 4.14%
----------------------------------------
